In [1]:
from analysis_helpers import *

folder = r'E:/CSHL/pita_g0/'
apfiles,nidqfile = list_spikeglx_files(folder)

# load the AP channel for each probe and extract the onsets and offsets from the binary files
aps = []
apmetas = []
aponsets = []
apoffsets = []

for file in tqdm(apfiles):
    b = load_spikeglx_binary(file)
    aps.append(b[0])
    apmetas.append(b[1])
    o,f = unpack_npix_sync(b[0][:,-1])
    # load onsets for each probe
    aponsets.append(o)
    apoffsets.append(f)
# load the NIDQ channel and extract the onsets/offsets
nidq,nidqmeta = load_spikeglx_binary(nidqfile)
nidqonsets,nidqoffsets = unpack_npix_sync(nidq[:,-1])

100%|██████████| 1/1 [03:30<00:00, 210.34s/it]


In [2]:
# Print how many onsets and offsets are in each probe
print('\nPROBES:')
for i,ons in enumerate(aponsets):
    for k in ons.keys():
        npulses = len(ons[k])
        print(f'\t- imec{i} has {npulses} on channel {k}')
# and on the daq digital channels
print('\nDAQ:')
for k in nidqonsets.keys():
    npulses = len(nidqonsets[k])
    print(f'\t - has {npulses} on channel {k}')


PROBES:
	- imec0 has 1138 on channel 6

DAQ:
	 - has 540 on channel 0
	 - has 1138 on channel 5


In [3]:
# by inspecting the outputs, we know that the sync is on channel 6 for the probe and on channel 5 for the DAQ
apsyncchannel = 6
nidqsyncchannel = 5
# now lets interpolate to the first probe
# interpolate data to the first probe 
from scipy.interpolate import interp1d
apcorrections = []
syncpulses = aponsets[0][apsyncchannel]/apmetas[0]['sRateHz'] # these are the pulses to interpolate to
# there is only one probe here so these are not useful
for pulses in aponsets:
    apcorrections.append(interp1d(pulses[apsyncchannel],
                                  syncpulses, fill_value='extrapolate')) # in samples
    
nidqcorrection = interp1d(nidqonsets[nidqsyncchannel],
                                  syncpulses, fill_value='extrapolate') # in samples





In [41]:
# Now that we know we can correct for the actual experiment sync and extract the optogenetics pulse
# the visual stimulus is in digital channel 0 (port0.0)
visual_stimulus_sync = nidqcorrection(nidqonsets[0])

# now lets read the analog channel, the nitime 
nitime = nidqcorrection(np.arange(0,len(nidq))) 
# find the onsets of the optogenetics stimulus
thresh = 200
# first threshold the analog signal (ideally we use a trigger signal instead of a threshold)
thresholded_analog = (nidq[:,0]>thresh).astype(np.float32)
# then do the difference, the onsets will be +1, the offsets will be -1
diffed_threshold = np.hstack([0,np.diff(thresholded_analog)])
# now find the location of the onsets and offsets
opto_stimulus_sync = nidqcorrection(np.where(diffed_threshold>0.5)[0])
# this will also detect cases when the input to the LED fluctuates a bit, lets filter that.
# We could either filter the signal or use info from the visual stimulus.
# Lets use info from the visual stimulus, we will only use pulses that occur 0.7s after the visual stim
filtered_opto = []
for o in visual_stimulus_sync:
    # for each trial
    tmp = opto_stimulus_sync[opto_stimulus_sync>o]
    if len(tmp):
        # if there is an opto that is smaller than the visual trial
        if (tmp[0]-o)<1:
            # if the first happended less than 1 ms in the visual stim include it. 
            filtered_opto.append(tmp[0])
opto_stimulus_sync = np.array(filtered_opto)

(21.675205325333334, 20.680526666666665)

In [47]:
%matplotlib qt

import pylab as plt
import numpy as np

# lets plot how far off we were for each data stream

# There is only one probe here so that is not useful
# for iprobe,pulses in enumerate(aponsets):
#     plt.figure(figsize=[8,3])
#     plt.plot((syncpulses-pulses[apsyncchannel]/apmetas[iprobe]['sRateHz'])*1000,'ko',label='measured',alpha=0.5)
#     plt.plot((syncpulses-apcorrections[iprobe](pulses[apsyncchannel]))*1000,'r.',label='corrected',alpha=0.5)
#     plt.ylim([-5,5])
#     plt.xlabel('time in experiment (s)')
#     plt.ylabel('measured-corrected (ms)')
# for here we care only about the NIDAQ
plt.figure()
plt.plot([syncpulses[0],syncpulses[-1]],[0,0],'-',color='lightgray')
plt.plot((syncpulses-nidqonsets[nidqsyncchannel]/nidqmeta['sRateHz'])*1000,'k.',label='measured',alpha=0.5)
plt.plot((syncpulses-nidqcorrection(nidqonsets[nidqsyncchannel]))*1000,'r.',label='corrected',alpha=0.5)
plt.xlabel('time in experiment (s)')
plt.ylabel('measured-corrected (ms)')
plt.ylim([-2,2]);

In [50]:
# lets plot the syncs 
plt.figure()
plt.plot(nitime[:1000000],nidq[:1000000,0])

plt.vlines(visual_stimulus_sync[visual_stimulus_sync<nitime[1000000]],15000,20000)
plt.vlines(opto_stimulus_sync[opto_stimulus_sync<nitime[1000000]],20000,25000,color='blue')

plt.yticks([7500,18000,23000],['optogenetics pulses','visual stimuli','opto stimuli']);



In [49]:
# save the syncs
import h5py as h5
outputfile = pjoin(folder,'experiment_syncs.h5')
with h5.File(outputfile,'w') as fd:
    #save the corrected syncs, the sampling rate, and the 
    fd.create_dataset('opto_sync_s',data = opto_stimulus_sync)
    fd.create_dataset('visual_sync_s',data = visual_stimulus_sync)
    fd.create_dataset('stream_apsyncs',data = syncpulses)
    fd.create_dataset('stream_nisyncs',data = nidqoffsets[nidqsyncchannel])
    fd.create_dataset('sampling_rate',data = apmetas[0]['sRateHz'])

In [51]:
opto_stimulus_sync

array([  14.68704   ,   20.68052667,   22.68037333,   24.68125519,
         26.68505333,   28.68094   ,   30.68074   ,   32.68170031,
         34.68050667,   38.68134512,   40.68311526,   44.6808318 ,
         46.68572   ,   54.68312667,   56.68096513,   58.68085333,
         60.68373333,   72.68190667,   76.68171333,   78.68147333,
         84.68132   ,   88.68488667,  100.68202   ,  106.68154855,
        108.68350667,  110.68634494,  112.68215333,  114.68203524,
        120.68272   ,  124.68432667,  126.68236667,  128.68221333,
        134.68166   ,  136.68174   ,  138.68346197,  142.68222667,
        144.68206667,  146.68687333,  148.68287333,  150.68271333,
        152.6846    ,  154.683     ,  156.68228   ,  162.68236667,
        166.68325333,  168.68217333,  172.68298   ,  174.68486   ,
        180.68602667,  184.68331333,  186.68319333,  188.68504   ,
        192.68272   ,  200.68309333,  202.68397333,  204.68333531,
        208.68258   ,  210.68454   ,  212.68330864,  218.68551